In [47]:
import pandas as pd
import numpy as np
import os
import math
import queue

Open Files

In [48]:
train_file_x_loc = 'data_1\\train\X_train.xlsx'
train_file_y_loc = 'data_1\\train\y_train.xlsx'
test_file_loc = 'data_1\Dataset1_test\X_test.xlsx'

train_data_x = pd.read_excel(train_file_x_loc)
train_data_y = pd.read_excel(train_file_y_loc)
test_data = pd.read_excel(test_file_loc)

Convert to numpy array

In [49]:
tr_x = train_data_x.values
tr_y = [i[0] for i in train_data_y.values]
test = test_data.values
num_features = len(tr_x[0])

Imputation

In [50]:
for cur_f in range(len(tr_x[0])):

    temp = [x for x in tr_x[:,cur_f] if np.isnan(x) == False]
    feature_mean = np.mean(temp)

    for i in range(len(tr_x)):
        if np.isnan(tr_x[i][cur_f]):
            tr_x[i][cur_f] = feature_mean

Entropy calculating function

In [51]:
labels = [3,4,5,6,7,8]

count = np.bincount(tr_y)[3:]

# Take bin count of each label value as input
def get_entropy(count):
    num_data = np.sum(count)
    total = 0

    for i in count:
        prob = i / num_data
        if prob == 0:
            continue
        total += prob * np.log2(prob)

    return -total

Class for the decision tree

In [52]:
class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None    # Feature values, including labels
        self.f_in = None    # The index of the traversal criteria
        self.thresh = None  # The value threshold of the traversal criteria
        self.label = None   # Label with the max amount
    
    def __init__(self, data):
        self.left = None
        self.right = None
        self.data = data
        self.f_in = None
        self.thresh = None
        self.label = None

Create Decision Tree:

In [53]:
cur_data = tr_x
temp = []

# Append target feature to cur_data
for i in range(len(cur_data)):
    temp.append( np.append(cur_data[i],int(tr_y[i])) )
cur_data = np.array(temp)

root = Node(cur_data)
cur_node = root

q = queue.Queue() # A queue for processing nodes of the decision tree
q.put(cur_node)


# # # LOOP

while q.empty() == False:

    cur_node = q.get()
    cur_data = cur_node.data

    # If data length is 0, continue
    if len(cur_data) == 0:
        continue
    
    cur_label = [int(temp) for temp in cur_data[:,num_features]]
    temp_count = np.bincount(cur_label)[3:]

    # Assign the selected label to the current node
    cur_node.label = np.argmax(temp_count) + 3

    # If all data has the same label, stop
    areAllLabelSame = False

    for temp in temp_count:
        if temp == sum(temp_count):
            areAllLabelSame = True
            break
    if areAllLabelSame:
        continue

    # We want to get (feature, val) that minimizes IG
    split_feat_in = 0 
    split_thresh = 0
    min_ent = np.Inf

    # Go through each feature, sort the values
    # i = current feature index
    for i in range(num_features):  # range(num_features)
        cur_feat_val = np.sort(cur_data[:,i])

        # Iterate through each sorted value 
        for j in range(len(cur_feat_val) - 1):

            # Get every unique feature values
            if (cur_feat_val[j] !=  cur_feat_val[j+1]):
                cur_thresh = cur_feat_val[j]

                # Split data into two based on cur_thresh, get two labels
                label1 = []
                label2 = []
                for temp_dat in cur_data:
                    if temp_dat[i] <= cur_thresh:
                        label1.append(int(temp_dat[num_features])) # Append the label
                    else:
                        label2.append(int(temp_dat[num_features]))
                
                label1 = np.array(label1)
                label2 = np.array(label2)

                count1 = np.bincount(label1)[3:]
                count2 = np.bincount(label2)[3:]

                num_labels_1 = np.sum(count1)
                num_labels_2 = np.sum(count2)

                w_avg_ent = (num_labels_1 / (num_labels_1 + num_labels_2)) * get_entropy(count1) + (num_labels_2 / (num_labels_1 + num_labels_2)) * get_entropy(count2)
                
                # Maximizing IG == minimizing w_avg_ent

                if w_avg_ent < min_ent:
                    min_ent = w_avg_ent
                    split_feat_in = i
                    split_thresh = cur_thresh
    
    # Split data into two
    data_left = []
    data_right = []

    # For each data put in either left / right
    for temp_dat in cur_data:
        if temp_dat[split_feat_in] <= split_thresh:
            data_left.append(temp_dat)
        else:
            data_right.append(temp_dat)
    
    data_left = np.array(data_left)
    data_right = np.array(data_right)


    # Update data structure
    cur_node.thresh = split_thresh
    cur_node.f_in = split_feat_in

    cur_node.left = Node(data_left)
    cur_node.right = Node(data_right)

    q.put(cur_node.left)
    q.put(cur_node.right)

    #print(cur_node.thresh)

Test Performance: 

In [54]:
print('Test Input: ')
print(test)

Test Input: 
[[9.10e+00 7.65e-01 4.00e-02 ... 3.29e+00 5.40e-01 9.70e+00]
 [6.90e+00 7.40e-01      nan ...      nan 6.30e-01 1.15e+01]
 [     nan 5.80e-01 2.30e-01 ... 3.21e+00 5.80e-01 9.50e+00]
 ...
 [9.00e+00 5.80e-01 2.50e-01 ... 3.23e+00 5.70e-01 9.70e+00]
 [8.30e+00 8.45e-01 1.00e-02 ... 3.32e+00 5.80e-01 1.10e+01]
 [8.60e+00 6.30e-01 1.70e-01 ... 3.09e+00 5.20e-01 9.30e+00]]


In [55]:
for cur_f in range(len(test[0])):

    temp = [x for x in test[:,cur_f] if np.isnan(x) == False]
    feature_mean = np.mean(temp)

    for i in range(len(test)):
        if np.isnan(test[i][cur_f]):
            test[i][cur_f] = feature_mean

In [56]:
i = 0
pred_label = []

for item in test:
    cur_node = root
    while True:
        if cur_node.left == None or cur_node.right == None:
            pred_label.append(cur_node.label)
            break

        if item[cur_node.f_in] <= cur_node.thresh:
            cur_node = cur_node.left
        else: 
            cur_node = cur_node.right
    
    i += 1

pred_label = np.array(pred_label)

d = pd.DataFrame({'class': pred_label})
df = pd.DataFrame(data=d)
df.to_excel('out_1.xlsx',index=False)
print(d)

     class
0        6
1        6
2        6
3        5
4        5
..     ...
251      5
252      6
253      5
254      5
255      5

[256 rows x 1 columns]
